In [1]:
from dsl.dependencies import get_property_model, register_dependencies
from dsl.utils.config_providers.client_config_provider import initialise_client_config, SupportedSource
from uuid import UUID
from dataclasses import dataclass, field
import os
import petl as etl

initialise_client_config(SupportedSource.Pigi3TextExtract)
register_dependencies()
pm = get_property_model()

In [2]:
#@dataclass
#class DateProperty:
#    guid: str
#    table: str
table_prop_is_a_well_triples = set()

# iterate property model
for prop in pm.properties:
    if prop['DataType'] != 'Dt':
        continue
        
    prop_guid = UUID(prop['Guid'])    
    is_a_well_property = prop['AnalysisGroup'] == 'well'
    tab_prefix = 'well' if is_a_well_property else ''
    
    if prop['FullName'] == 'Analysis date':
        table = "AnalysisMetadata"
    else:
        table = f'{tab_prefix}Observation{prop_guid.hex}'
    
    table_prop_is_a_well_triples.add((table, prop['ShortName'], is_a_well_property))
    

In [3]:
# generate queries
for table, prop, is_a_well in sorted(table_prop_is_a_well_triples):
    if table == 'AnalysisMetadata':
        query = f"select AnalysisDate as [Analysis.{prop}] from AnalysisMetadata where AnalysisDate like '%23:%';"
    else:
        query = f"select Result as [{'Well' if is_a_well else 'Sam'}.{prop}] from {table} where Result like '%23:%';"
    print(query)

select AnalysisDate as [Analysis.Date] from AnalysisMetadata where AnalysisDate like '%23:%';
select Result as [Sam.Sampling date] from Observatione178655787f94ce986d180a0eaf35478 where Result like '%23:%';
select Result as [Well.Start date] from wellObservation2aaaf960392d409993b205d210416cfb where Result like '%23:%';
select Result as [Well.Compl date] from wellObservationd69c057cd9f14682b0796b34d84a0d02 where Result like '%23:%';
select Result as [Well.End date] from wellObservationf6aeeadc69a94b75ad911a4cc5f043c0 where Result like '%23:%';


In [4]:
# generate updates
# note: tried format(dateadd(hour, 1, parse(AnalysisDate as datetime)), 'yyyy-MM-dd HH:mm:ssZ') - this added two hours
#       just parsing seems to add an our (perhaps it thinks the input is local and +1 -> TUC for daylight saving)
for table, _, _ in sorted(table_prop_is_a_well_triples):
    if table == 'AnalysisMetadata':
        query = f"update {table}\n   set AnalysisDate = format(parse(AnalysisDate as datetime), 'yyyy-MM-dd HH:mm:ssZ')\n where AnalysisDate like '%23:%';\n"
    else:
        query = f"update {table }\n   set Result = format(parse(Result as datetime), 'yyyy-MM-dd HH:mm:ssZ')\n where Result like '%23:%';\n"
    print(query)

update AnalysisMetadata
   set AnalysisDate = format(parse(AnalysisDate as datetime), 'yyyy-MM-dd HH:mm:ssZ')
 where AnalysisDate like '%23:%';

update Observatione178655787f94ce986d180a0eaf35478
   set Result = format(parse(Result as datetime), 'yyyy-MM-dd HH:mm:ssZ')
 where Result like '%23:%';

update wellObservation2aaaf960392d409993b205d210416cfb
   set Result = format(parse(Result as datetime), 'yyyy-MM-dd HH:mm:ssZ')
 where Result like '%23:%';

update wellObservationd69c057cd9f14682b0796b34d84a0d02
   set Result = format(parse(Result as datetime), 'yyyy-MM-dd HH:mm:ssZ')
 where Result like '%23:%';

update wellObservationf6aeeadc69a94b75ad911a4cc5f043c0
   set Result = format(parse(Result as datetime), 'yyyy-MM-dd HH:mm:ssZ')
 where Result like '%23:%';

